In [1]:
script_path = getwd()                                # %exclude_jupyterlab%
script_path                                          # %exclude_jupyterlab%

[1] "/Volumes/ccrsf-ifx/Software/github/SF_scMaestro/workflow/scripts/rna"

In [2]:
outdir = "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/clusterprofiler/" # %exclude_jupyterlab%
path = "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/seurat/"            # %exclude_jupyterlab%

In [3]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_clusterprofiler_opt.R --path=', path, ' --outdir=', outdir,'--genome="hg38" '),  # %exclude_jupyterlab%
       intern = T) # %exclude_jupyterlab%

character(0)

In [6]:
library(Seurat)
library(clusterProfiler)
library(dplyr)

In [7]:
getwd()                                         # %exclude_jupyterlab%
setwd(outdir)                                   # %exclude_jupyterlab%
getwd()

[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/clusterprofiler"

[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/clusterprofiler"

In [8]:
opt = readRDS("./opt.rds")                                 # %exclude_jupyterlab%

In [9]:
opt                                                              

$path
[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/seurat/"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$outdir
[1] "/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test4pl_outdir_3sample_subset1/NyallLondon_CS032345_16scRNASeqTCRLib_081122_rna_full_default/Analysis/1_51_IP_10_5GEX/clusterprofiler/"

$help
[1] FALSE

In [10]:
setwd(opt$outdir)                       

In [11]:
seuratPath = opt$path
genome     = opt$genome


In [12]:
if (genome == "mm10") {
  library(org.Mm.eg.db)
  orgdb = "org.Mm.eg.db"
  organism = "mmu"
}else if(genome == "hg38"){
  library(org.Hs.eg.db)
  orgdb = "org.Hs.eg.db"
  organism = "hsa"
}

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVigne

In [13]:
paths <- Sys.glob(paste(seuratPath, '/markers_res*.rds', sep=""))

resolutions <- sub(".rds", '', sub(paste(seuratPath,'/markers_res', sep=""),'',paths))

markers <- vector("list")
for (res in resolutions){
  markers[[res]] <- readRDS(paste(seuratPath, "/markers_res", res, ".rds", sep=""))
}

In [14]:
for (res in resolutions) {
  degs <- markers[[res]]

  maxGenes <- 100
  categories <- 50
  clusters <- unique(degs$cluster)

  #up <- degs[degs["avg_logFC"] > 0,]
  #down <- degs[degs["avg_logFC"] < 0,]
  temp <- degs %>% group_by(cluster) %>% top_n(-maxGenes, p_val)

  result <- vector("list")
  for(i in clusters){
    result[i] <- list(temp$gene[temp$cluster==i])
  }

  ids = list()
  for (i in clusters) {
        try({
                test <- bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb)
                ids[paste("X", i, sep = "")] <- list(test$ENTREZID)
        })
  }

  try({
    ck <- compareCluster(geneCluster = ids, fun = "enrichKEGG", organism=organism)#, OrgDb = "org.Mm.eg.db")
    ht = max(7, (min(ceiling(sum(ck@compareClusterResult$Count > 0)/4), ceiling(categories*length(clusters)/4))))
    wt = max(ceiling(max(nchar(ck@compareClusterResult$Description))/8+length(ids)/4), 7)
    png(paste("enrichKegg_", res, ".png", sep=""), height=ht, width=wt, units='in', res=300)
    print(dotplot(ck, showCategory=categories))
    dev.off()
  })
  try({
    ck <- compareCluster(geneCluster = ids, fun = "enrichGO", OrgDb = orgdb)
    ht = max(7, min(ceiling(sum(ck@compareClusterResult$Count > 0)/4), ceiling(categories*length(clusters)/4)))
    wt = max(ceiling(max(nchar(ck@compareClusterResult$Description))/8), 7)
    png(paste("enrichGO_", res, ".png", sep=""), height=ht, width=wt, units='in', res=300)
    print(dotplot(ck, showCategory=categories))
    dev.off()
  })
  try({
    ck <- compareCluster(geneCluster = ids, fun = "groupGO", ont="BP", OrgDb = orgdb)
    #ht = max(7, min(ceiling(sum(ck@compareClusterResult$Count > 0)/4), ceiling(categories*length(clusters)/4)))
    wt = max(ceiling(max(nchar(as.character(ck@compareClusterResult$Description)))/8+length(ids)/4), 7)
    png(paste("groupGO_", res, ".png", sep=""), height=7, width=wt, units='in', res=300)
    print(dotplot(ck, showCategory=categories))
    dev.off()
  })
}

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“2% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“2% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“19% of i

Error in file(con, "r") : 
  cannot open the connection to 'https://rest.kegg.jp/link/hsa/pathway'
Error in geom_point(aes_string(fill = colorBy)) : 
  Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error in `compute_aesthetics()`:
! Aesthetics are not valid data columns.
✖ The following aesthetics are invalid:
✖ `fill = p.adjust`
ℹ Did you mistype the name of a data column or forget to add `after_stat()`?


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“4% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“2% of in

Error in file(con, "r") : 
  cannot open the connection to 'https://rest.kegg.jp/link/hsa/pathway'
Error in geom_point(aes_string(fill = colorBy)) : 
  Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error in `compute_aesthetics()`:
! Aesthetics are not valid data columns.
✖ The following aesthetics are invalid:
✖ `fill = p.adjust`
ℹ Did you mistype the name of a data column or forget to add `after_stat()`?


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“4% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“4% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“2% of in

Error in file(con, "r") : 
  cannot open the connection to 'https://rest.kegg.jp/link/hsa/pathway'
Error in geom_point(aes_string(fill = colorBy)) : 
  Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error in `compute_aesthetics()`:
! Aesthetics are not valid data columns.
✖ The following aesthetics are invalid:
✖ `fill = p.adjust`
ℹ Did you mistype the name of a data column or forget to add `after_stat()`?


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“4% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“4% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = orgdb):
“1% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(result[[i]], fromType 

Error in file(con, "r") : 
  cannot open the connection to 'https://rest.kegg.jp/link/hsa/pathway'
Error in geom_point(aes_string(fill = colorBy)) : 
  Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error in `compute_aesthetics()`:
! Aesthetics are not valid data columns.
✖ The following aesthetics are invalid:
✖ `fill = p.adjust`
ℹ Did you mistype the name of a data column or forget to add `after_stat()`?


In [18]:
getOption("clusterProfiler.download.method")

[1] "libcurl"

In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] org.Hs.eg.db_3.18.0    AnnotationDbi_1.64.1   IRanges_2.34.1        
 [4] S4Vectors_0.38.2       Biobase_2.60.0         BiocGenerics_0.46.0   
 [7] dpl